In [ ]:
import torch
import plotly.graph_objs as go
from transformers import AutoModel

def compute_and_visualize_param_difference(original_model_name, finetuned_model_name, param_name):
    """
    Compute and visualize the differences in a specific parameter between two models, applicable to the Qwen2 model.
    
    Parameters:
        original_model_name (str): Original Hugging Face model name.
        finetuned_model_name (str): Fine-tuned Hugging Face model name.
        param_name (str): The name of the parameter to compare (e.g., 'model.layers.0.self_attn.q_proj.weight').
    """
    # Load the original and fine-tuned models
    original_model = AutoModel.from_pretrained(original_model_name)
    finetuned_model = AutoModel.from_pretrained(finetuned_model_name)
    
    original_model.eval()
    finetuned_model.eval()
    
    # Get the specific parameter
    original_param = original_model.state_dict()[param_name]
    finetuned_param = finetuned_model.state_dict()[param_name]
    
    # Compute the parameter difference
    param_diff = finetuned_param.detach().cpu().numpy() - original_param.detach().cpu().numpy()
    
    # Create a heatmap of the differences
    fig = go.Figure(data=go.Heatmap(
        z=param_diff,
        colorscale='RdBu',  # Red-blue gradient: red indicates positive difference, blue indicates negative difference
        colorbar=dict(title="Difference")
    ))

    # Set title and labels
    fig.update_layout(
        title=f"Difference between {original_model_name} and {finetuned_model_name} - {param_name}",
        xaxis_title="Units",
        yaxis_title="Units",
        xaxis=dict(scaleanchor="y"),  # Keep the aspect ratio square
        yaxis=dict(scaleanchor="x")
    )

    # Display the image
    fig.show()

if __name__ == "__main__":
    # Example usage
    original_model_name = "Qwen/Qwen2-7B-Instruct"
    finetuned_model_name = "path/to/your/finetuned/model"  # Replace with the path or name of your fine-tuned model
    param_name = "model.layers.0.self_attn.q_proj.weight"  # Replace with the parameter name you wish to compare
    
    compute_and_visualize_param_difference(original_model_name, finetuned_model_name, param_name)
